* 쿠팡와우,마켓컬리,SSG 새벽심야배송하는 3개업체 비교,각 업체마다 특징이 나타나는지,제품의 특징이 나타나는지

In [1]:
pip install selenium 

Note: you may need to restart the kernel to use updated packages.


In [1]:
def insta_searching(word):
    url = "https://www.instagram.com/explore/tags/" + word
    return url

from selenium import webdriver

driver = webdriver.Chrome("chromedriver.exe")

import time

In [4]:
#인스타 접속
driver.get('https://www.instargram.com')
time.sleep(5) #시간을 넉넉히 설정합니다.

In [7]:
#로그인에 필요한 사항을 브라우저 제어를통해 입력합니다.
email = '01086897540'   ### 사용할 계정 정보 입력
input_id = driver.find_elements_by_css_selector('input._2hvTZ.pexuQ.zyHYP')[0]
input_id.clear()
input_id.send_keys(email)

password = 'rkdgus12!@' ### 비번 정보 수정
input_pw = driver.find_elements_by_css_selector('input._2hvTZ.pexuQ.zyHYP')[1]
input_pw.clear()
input_pw.send_keys(password)
input_pw.submit()
time.sleep(3)


In [8]:
word = "롯데월드" #검색할 해시태그. 띄어쓰기를 사용하면 안됩니다
url = insta_searching(word)
driver.get(url)
time.sleep(2)

In [9]:
#첫게시물을 여는 함수를 정의합니다
def select_first(driver):
    first = driver.find_element_by_css_selector("div._9AhH0")
    first.click()
    time.sleep(3)

In [10]:
import re
from bs4 import BeautifulSoup

#내용을 가져오는 함수를 정의합니다.
def get_content(driver):
    
    html = driver.page_source
    soup = BeautifulSoup(html, 'lxml')
    

    try:
        content = soup.select('div.C4VMK > span')[0].text
    except:
        content = ' '
    # 정규표현식을 활용하여 해시태그를 가져옵니다.
    tags = re.findall(r'#[^\s#,\\]+', content)  
    # 작성일자를 가져옵니다.
    date = soup.select('time._1o9PC.Nzb55')[0]['datetime'][:10]
    # 좋아요 수를 가져옵니다.
    try:
        like = soup.select('div.Nm9Fw > button')[0].text[4:-1]   
    except:
        like = 0
    # 위치정보를 가져옵니다.
    try: 
        place = soup.select('div.M30cS')[0].text
    except:
        place = ''
    # 수집한 정보를 리스트로 저장합니다.
    data = [content, date, like, place, tags]
    return data



In [11]:
#다음게시글을 여는 함수를 정의합니다.
def move_next(driver):

    right = driver.find_element_by_css_selector ('a.coreSpriteRightPaginationArrow')
    right.click()
    time.sleep(3)

In [12]:
#함수를 차례대로 실행합니다.
select_first(driver)
results = [ ]
target = 300000      # 크롤링할 게시글 수
for i in range(target):
    # 게시글 수집에 오류 발생시 2초 대기후 다음글로 넘어갑니다.
    try:
        data = get_content(driver)    # 게시글 정보 가져오기
        results.append(data)
        move_next(driver)
    except:
        time.sleep(2)
        move_next(driver)


NoSuchElementException: Message: no such element: Unable to locate element: {"method":"css selector","selector":"a.coreSpriteRightPaginationArrow"}
  (Session info: chrome=89.0.4389.114)


In [33]:
#result적당히 출력해보기
print(results[:2])




[['2021.03.15.월-#아보카도김치볶음밥#치즈돈까스#해쉬브라운-세상 맛있어서 이 조합으로 한 번 더 해먹기 향 찐한 아보카도까지 올려주니 💛🥑💛치즈돈까스는 내사랑 #마켓컬리 에서샀는데모짜렐라&체다 치즈가 듬뿍 들어있어요즘 빼놓지 않고 주문하는 중 (매우 추천)', '2021-03-15', 0, '', ['#아보카도김치볶음밥', '#치즈돈까스', '#해쉬브라운-세상', '#마켓컬리']], ['#여니식단_점심⠀- 흑미밥 1/2공기- 계란2알 + 체다치즈 1/2 + 크래미 1개- #존쿡델리미트 비엔나- 김치찜 3점- 멀티그레인 무화과빵 1조각- #모모스커피 3모금⠀점심은 집에 있는 재료로호다닥 차려먹었어요 :)⠀근데 존쿡델리미트 제품들은 전부 짜다는 걸제가 깜박했지 뭐에요...⠀뜨거운 물에 잠시 담가놨는데도저한테는 짜던 비엔나씨...⠀전체적으로 밥량에 비해 짜서입가심으로 빵이랑 커피쓰를 먹어줬어요!⠀저희 집은 근처에 있는 모모스 커피에서항상 드립커피를 사다먹는데이제 다른 커피는 못 먹는 입이 되버린..ㅎㅎ⠀커피 좋아하시는 분들은#마켓컬리 에서도 팔고 있으니한 번 드셔보세용♥️⠀그럼 오늘도 행복한 하루 되시길 바라며우리 저녁에 다시 만나용 🙌🏻⠀', '2021-02-28', 0, 'Busan 釜山, South Korea', ['#여니식단_점심⠀-', '#존쿡델리미트', '#모모스커피', '#마켓컬리']]]


In [60]:
pd.DataFrame(results)

,0,1,2,3,4
0,300일 촬영 망...😭.11월에 스튜디오 촬영ᄒ...,2020-10-14,0,치앙마이방콕,"[#아들아왜자꾸우니울지마라.., #울보재윤, #..."
1,돈마루사장님 짱!!!!!!! 🥩 고기맛 저도 보자...,2020-12-24,0,,"[#한돈, #돈마루, #쿠팡새벽배송, #돼지고기, #..."
2,.오전 간식은 #던킨도너츠오늘 점심은 #잔치국수아직 12시네😆#홈쿡#요리#아이밥상#...,2020-12-02,0,,"[#던킨도너츠오늘, #잔치국수아직, #홈쿡, #요리, #아이밥상, #간단메뉴, #집..."
3,.유치원에서의 HBD🎂🎁🎀.일끝나고벼락치기로오후 늦게가볍게 준비하려던게.이래저래 수...,2020-11-15,0,,"[#내사랑이호호준👶, #생일파티, #유치원친구들, #행복한하루, #빼빼로데이, #엄..."
4,오늘 저녁 닭한마리 칼국수~친구추천으로쿠팡에서 시켰는데와 진짜 식당에서 먹는것처럼 ...,2021-01-08,0,다산신도시,"[#일상, #주부일상, #데일리, #좋아요, #주부그램, #저녁, #저녁시간, #저..."
...,...,...,...,...,...
1065,#밤에올리는아침밥네네. 바쁩니다. 뭐하고 살고 있는지 모를 만큼 바쁩니다. 그래서 ...,2018-11-02,0,,"[#밤에올리는아침밥네네., #쿠팡, #새벽배송, #오이고추, #가지, #부추, #4..."
1066,쿠팡새벽배송♡어제밤11시넘어서 처음 주문해봤는데 오늘새벽에 바로 문앞에 똭~😀 계란...,2018-11-01,0,,"[#쿠팡새벽배송, #좋은세상이다, #대박나세요]"
1067,내맘대로..클렌즈 주스...ㅋㅋ 케일+사과피도 맑게 해준다 하고...화장실도 잘가게...,2018-10-30,0,구의동우리집,"[#클렌즈주스, #휴롬주스, #케일주스, #케일, #쿠팡새벽배송, #몸에좋은, #내..."
1068,#황금팽이버섯#버섯#쿠팡새벽배송맛난요리할게요~~^^,2018-10-27,0,,"[#황금팽이버섯, #버섯, #쿠팡새벽배송맛난요리할게요~~^^]"


In [13]:
#데이터프레임 만들고 엑셀로 저장하기
import pandas as pd

results_df = pd.DataFrame(results)
results_df.columns = ['content','data','like','place','tags']
results_df.to_excel('crawling_sample_l_w.xlsx', index=False) 

In [24]:
results_df[results_df['tags']==]

ValueError: ('Lengths must match to compare', (10474,), (0,))

In [30]:
results_df['tags']==""

0        False
1        False
2        False
3        False
4        False
         ...  
10469    False
10470    False
10471    False
10472    False
10473    False
Name: tags, Length: 10474, dtype: bool

In [28]:
results_df['tags'].iloc[1]

[]